# Import

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
from gensim.models.word2vec import Word2Vec
import gc
import time
import pickle

# Load graph

In [2]:
with tf.gfile.FastGFile(os.path.join(
    '../../../data/inception-2015-12-05/classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
g = tf.get_default_graph()

## Create own branch

In [3]:
with tf.name_scope('own'):
    y = tf.placeholder("float")
    x = g.get_tensor_by_name('pool_3/_reshape:0')
    x_batch = g.get_tensor_by_name("pool_3:0")
    w = tf.Variable(tf.random_normal(
        [int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=float('1e-5')).minimize(cost)

def mini_batch(sess, y_batch, im_batch, batch_size):
#with tf.name_scope('batch'):
    y_batch = tf.placeholder("float", shape=(batch_size, 300))
    im_batch = tf.placeholder("float", shape=(batch_size, None, None, 3))
    cost_batch = 0
    for j in range(np.shape(y_batch)[0]):
        temp_im = g.get_tensor_by_name('DecodeJpeg/contents:0')
        temp_im = im_batch[j,:]
        cost_batch += sess.run(cost,{'DecodeJpeg/contents:0':im_batch[j,:,:], y:y_batch[j,:]})
    batch_optimizer = tf.train.GradientDescentOptimizer(learning_rate=float('1e-5')).minimize(cost_batch)
    return batch_optimizer

with tf.name_scope('try_values'):
    xxx = g.get_tensor_by_name('DecodeJpeg/contents:0')
    print(xxx, end='\n\n')
    print(xxx.consumers())
    yyy = g.get_tensor_by_name('DecodeJpeg/contents:0')
    print(yyy)
    print(xxx)

# Read data

In [4]:
# Images
image_path = '../../../data/train2014/'
image_list = os.listdir(image_path)
# Caption dictionaries
vec_dict = np.load('../../../data/word2vec_train.npy').item()
with open('../../../data/caption_train.pkl', 'rb') as f:
    string_dict = pickle.load(f)

# Train model

In [31]:
from PIL import Image
with tf.Session(graph=g) as sess:
#    # Initialize writers
#    timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
#    train_writer = tf.summary.FileWriter('./sums/train/'+timestamp, graph=g, flush_secs=20)
#    test_writer = tf.summary.FileWriter('./sums/test/'+timestamp, graph=g, flush_secs=20)
    
    sess.run([w.initializer, b.initializer])
    for i, im in enumerate(image_list[:500]):
        # Tried to follow
        # http://stackoverflow.com/questions/35274457/inceptionv3-and-transfer-learning-with-tensorflow/40709836#40709836
        image_1 = np.array(Image.open(image_path + image_list[0]))
        image_2 = np.array(Image.open(image_path + image_list[1]))
        batched_images = np.stack((image_1[:299,:299,:],image_2[:299,:299,:]))
        input(sess.run(x_batch, {'ResizeBilinear:0':batched_images}))
        # Choose one of the five captions randomly
        r = random.randrange(len(vec_dict[im]))
        y_temp = vec_dict[im][r].reshape((1,300))

        optimizer.run(feed_dict={'DecodeJpeg/contents:0':image, y:y_temp})
        train_writer.add_summary(sess.run(cost_summary,{'DecodeJpeg/contents:0':image, y:y_temp}), i)
        print(i, end=' ')
        
        if i % 10 == 0:
            test_image = tf.gfile.FastGFile(image_path + image_list[-1], 'rb').read()
            y_test = vec_dict[image_list[-1]][1].reshape((1,300))
            test_writer.add_summary(sess.run(cost_summary,{'DecodeJpeg/contents:0':test_image, y:y_test}), i)

Tensor("own/weights/read:0", shape=(2048, 300), dtype=float32)
Tensor("own/bias/read:0", shape=(1, 300), dtype=float32)


ValueError: Cannot feed value of shape (2, 299, 299, 3) for Tensor 'ResizeBilinear:0', which has shape '(1, 299, 299, 3)'